# The GEE_ISMN Package

## Table of Contents
1. Install & Import modules
2. Setup
3. Preprocessing
4. Google Earth Engine (GEE)
5. Postprocessing
6. Visualization

## 1. Install & Import modules

In [4]:
import sys
!{sys.executable} -m pip install ismn
!{sys.executable} -m pip install geehydro

c:\users\marco\miniconda3\python.exe: can't open file 'pip': [Errno 2] No such file or directory


In [5]:
conda install -c conda-forge earthengine-api

Solving environment: ...working... done

## Package Plan ##

  environment location: c:\users\marco\miniconda3

  added / updated specs:
    - earthengine-api


The following packages will be UPDATED:

  earthengine-api                              0.1.210-py_0 --> 0.1.214-py_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [6]:
from GEE_ISMN import setup_pkg as pkg
from GEE_ISMN import preprocess as prep
from GEE_ISMN import earthengine as earth
from GEE_ISMN import postprocess as post
from GEE_ISMN import visualization as vis

## 2. Setup
Before the start, a few things need to be set up. The ISMN datasets which should be analysed needs to be in a subdirectory of the working directory called ./data/ISMN. 
The function setup_pkg() checks if the directory exists and creats a new directory called ISMN_Filt in the data directory where the filltered ISMN files will be stored.
Also it will initialize the Google Earth Engine (GEE) API and checks if the credentials are existing. If not, the function will ask to authenticate.
At last you will be ask if you want to extract backscatter values for the point coordinates of each station (Option 0) or if you want calculated mean backscatter values for a bounding box of each location (Option 1).

In [7]:
user_input = pkg.setup_pkg()

Directory  ./data/ISMN_Filt/  already exists.
Do you want to extract... 
 backscatter values for the pixel coordinate (input: 0) 
 or the mean backscatter value for a box surrounding the pixel coordinate (input: 1)?0


## Preprocessing
In the preprocessing, the data is checked if it contains soil moisture measurements and will be filtered for a specific measurement depth which can be set as a parameter in the data_handling() function. Standardized the measurment depth will be 0.05 m. 

In [8]:
prep.data_handling(measurement_depth=0.05)

226 ISMN files were found in './data/ISMN/'. 
66 ISMN files with a measurement depth of 0.05 were copied to './data/ISMN_Filt/'


The filtered data can than be saved to a dictionary with the function data.import(). It will write an entry for every station and sensor used. 
Also the function creates an CSV-File "stations.csv" which contains the coordinates and the name of each entry. 

In [9]:
data_dict = prep.data_import()

## 4. Google Earth Engine
The filtered data will than be filtered again, this time by landcover type. Therefore the data_dict contains the data and the input_dict the user inputs from the setup. The landcover types can be set through the parameter landcover_ids. Standardized are the values 40 (clutivated and managed vegetation / agriculture) and 60 (Bare / sparse vegetation). <br>
The landcover ids for the different landcovers can be retrieved [here](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V_Global).

In [11]:
data_dict = earth.lc_filter(data_dict, user_input, landcover_ids=None)

26 out of 66 locations remain after applying the land cover filter.


Now we got only the locations we are interested in. The next step is now to retrieve the backscatter values for these locations. 

In [ ]:
data_dict = earth.get_s1_backscatter(data_dict)

## 5. Postprocessing

Because there are just limited recordings of Sentinel-1 data and much more soil moisture data, just the soil moisture value immediatley before the Sentinel-1 recording will be kept.

In [ ]:
data_dict = post.filter_s1(data_dict)

## 6. Visualization

For visualization, we need the station names which were created in preprocessing and saved to the CSV-File "stations.csv". The name is a combination of the Network, the station and the sensor.

In [ ]:
station_name = 'REMEDHUS-Guarrati-Stevens-Hydra-Probe'

In [ ]:
vis.show_map(data_dict, station_name)
img = vis.show_s1(data_dict, station_name, date)

With the function plot_data() it is possible to plot the specific data of a station with the needed orbit or polarization. It will show a Graph in which the ISMN soil moisture data is plotted against the Sentinel-1 backscatter values over time. <br>
The standard for the orbit will be "desc" for descending, but can also be choosen as "asc" for ascending.<br>
For the polarisation the options are "VH" and "VV".

In [ ]:
vis.plot_data(data_dict, station_name, orbit=None, pol=None)